In [1]:
import pandas as pd
import numpy as np

In [2]:
#DATASET DESCRIPTION https://www.kaggle.com/sakshigoyal7/credit-card-customers/code?datasetId=982921&sortBy=voteCount
BankRaw=pd.read_csv("BankChurners.csv")

In [3]:
BankRaw.head()

CLIENTNUM     Attrition_Flag  Customer_Age Gender  Dependent_count  \
0  768805383  Existing Customer            45      M                3   
1  818770008  Existing Customer            49      F                5   
2  713982108  Existing Customer            51      M                3   
3  769911858  Existing Customer            40      F                4   
4  709106358  Existing Customer            40      M                3   

  Education_Level Marital_Status Income_Category Card_Category  \
0     High School        Married     $60K - $80K          Blue   
1        Graduate         Single  Less than $40K          Blue   
2        Graduate        Married    $80K - $120K          Blue   
3     High School        Unknown  Less than $40K          Blue   
4      Uneducated        Married     $60K - $80K          Blue   

   Months_on_book  ...  Credit_Limit  Total_Revolving_Bal  Avg_Open_To_Buy  \
0              39  ...       12691.0                  777          11914.0   
1              44  ...        8256.0                  864           7392.0   
2              36  ...        3418.0                    0           3418.0   
3              34  ...        3313.0                 2517            796.0   
4              21  ...        4716.0                    0           4716.0   

   Total_Amt_Chng_Q4_Q1  Total_Trans_Amt  Total_Trans_Ct  Total_Ct_Chng_Q4_Q1  \
0                 1.335             1144              42                1.625   
1                 1.541             1291              33                3.714   
2                 2.594             1887              20                2.333   
3                 1.405             1171              20                2.333   
4                 2.175              816              28                2.500   

   Avg_Utilization_Ratio  \
0                  0.061   
1                  0.105   
2                  0.000   
3                  0.760   
4                  0.000   

   Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1  \
0                                           0.000093                                                                                    
1                                           0.000057                                                                                    
2                                           0.000021                                                                                    
3                                           0.000134                                                                                    
4                                           0.000022                                                                                    

   Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2  
0                                            0.99991                                                                                   
1                                            0.99994                                                                                   
2                                            0.99998                                                                                   
3                                            0.99987                                                                                   
4                                            0.99998                                                                                   

[5 rows x 23 columns]

In [4]:
BankRaw['Attrition_Flag'].value_counts()

Existing Customer    8500
Attrited Customer    1627
Name: Attrition_Flag, dtype: int64

In [5]:
#DEFINE CHURN TARGET
ChurnTarget=(BankRaw['Attrition_Flag']=='Attrited Customer').astype('int64')

Check data type and count

In [6]:
BankRaw.dtypes.value_counts()

int64      10
float64     7
object      6
dtype: int64

In [7]:
#BankRawDataType=BankRaw.dtypes

In [8]:
#SEPERATE VARIABLES BY DATA TYPE
BankNumerical=BankRaw.select_dtypes(include=['int64', 'float64'])
BankCategorical=BankRaw.select_dtypes(include=['object'])

In [9]:
list(BankNumerical), BankNumerical.head()

(['CLIENTNUM',
  'Customer_Age',
  'Dependent_count',
  'Months_on_book',
  'Total_Relationship_Count',
  'Months_Inactive_12_mon',
  'Contacts_Count_12_mon',
  'Credit_Limit',
  'Total_Revolving_Bal',
  'Avg_Open_To_Buy',
  'Total_Amt_Chng_Q4_Q1',
  'Total_Trans_Amt',
  'Total_Trans_Ct',
  'Total_Ct_Chng_Q4_Q1',
  'Avg_Utilization_Ratio',
  'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
  'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
    CLIENTNUM  Customer_Age  Dependent_count  Months_on_book  \
 0  768805383            45                3              39   
 1  818770008            49                5              44   
 2  713982108            51                3              36   
 3  769911858            40                4              34   
 4  709106358            40                3              21   
 

In [10]:
#REMOVE ID AND UNKNOWN VALUES
BankNumerical=BankNumerical.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'
                   , 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'
                    , 'CLIENTNUM'], axis=1)


In [11]:
#FILL UP MISSING VALUES
#STEP 1 - CHECK WHICH COLUMNS CONTAIN ANY MISSING VALUES

BankNumerical.isnull().sum(), BankCategorical.isnull().sum()

(Customer_Age                0
 Dependent_count             0
 Months_on_book              0
 Total_Relationship_Count    0
 Months_Inactive_12_mon      0
 Contacts_Count_12_mon       0
 Credit_Limit                0
 Total_Revolving_Bal         0
 Avg_Open_To_Buy             0
 Total_Amt_Chng_Q4_Q1        0
 Total_Trans_Amt             0
 Total_Trans_Ct              0
 Total_Ct_Chng_Q4_Q1         0
 Avg_Utilization_Ratio       0
 dtype: int64, Attrition_Flag     0
 Gender             0
 Education_Level    0
 Marital_Status     0
 Income_Category    0
 Card_Category      0
 dtype: int64)

In [12]:
#THERE IS NO MISSING DATA SO NO BACKFILL REQUIRED
#CONVERT CATEGORICAL VARIABLES TO DUMMY VARIABLES

from sklearn.preprocessing import OneHotEncoder

In [13]:

BankCategorical=BankCategorical.drop('Attrition_Flag', axis=1)
CatEncoder=OneHotEncoder(sparse=False)
BankCatOnehot=CatEncoder.fit_transform(BankCategorical)

In [14]:
BankCatOnehot.shape

(10127, 23)

In [15]:
#VARIABLES AND LABELS
BankCatOnehot, CatEncoder.categories_

(array([[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 1.]]),
 [array(['F', 'M'], dtype=object),
  array(['College', 'Doctorate', 'Graduate', 'High School', 'Post-Graduate',
         'Uneducated', 'Unknown'], dtype=object),
  array(['Divorced', 'Married', 'Single', 'Unknown'], dtype=object),
  array(['$120K +', '$40K - $60K', '$60K - $80K', '$80K - $120K',
         'Less than $40K', 'Unknown'], dtype=object),
  array(['Blue', 'Gold', 'Platinum', 'Silver'], dtype=object)])

In [16]:
#STANDARDISE THE NUMERIC VARIABLES
from sklearn.preprocessing import StandardScaler

In [17]:
NumericScaler=StandardScaler()
BankNumericStandardised=NumericScaler.fit_transform(BankNumerical)

In [18]:
BankNumericStandardised.shape

(10127, 14)

In [19]:
BankNumericStandardised

array([[-1.65405580e-01,  5.03368127e-01,  3.84620878e-01, ...,
        -9.73895182e-01,  3.83400260e+00, -7.75882235e-01],
       [ 3.33570383e-01,  2.04319867e+00,  1.01071482e+00, ...,
        -1.35734038e+00,  1.26085729e+01, -6.16275655e-01],
       [ 5.83058365e-01,  5.03368127e-01,  8.96451285e-03, ...,
        -1.91120566e+00,  6.80786367e+00, -9.97154993e-01],
       ...,
       [-2.90149571e-01, -1.03646242e+00,  8.96451285e-03, ...,
        -2.07004789e-01,  4.44305026e-01, -9.97154993e-01],
       [-2.03656544e+00, -2.66547146e-01,  8.96451285e-03, ...,
        -1.21794745e-01,  4.10696270e-02, -9.97154993e-01],
       [-4.14893562e-01, -2.66547146e-01, -1.36844216e+00, ...,
        -1.64399767e-01, -2.65557291e-01, -3.11572183e-01]])

Data visualisations

In [20]:
from pandas.plotting import scatter_matrix

In [21]:
ScatterPlot=scatter_matrix(BankNumerical[['Customer_Age','Total_Revolving_Bal',
                                          'Total_Trans_Amt', 'Total_Trans_Ct']], figsize=(12, 8))

In [22]:
#CREATE TRAINING AND TESTING DATASETS
from sklearn.model_selection import train_test_split

In [23]:
BankProcessed=np.concatenate((BankCatOnehot, BankNumericStandardised), axis=1)

In [26]:
XTrain, XTest=train_test_split(BankProcessed, test_size=0.2, random_state=61)
YTrain, YTest=train_test_split(ChurnTarget, test_size=0.2, random_state=61)
YTrain=np.array(YTrain)
YTest=np.array(YTest)

In [27]:
XTrain.shape, XTest.shape, YTrain.shape, YTest.shape

((8101, 37), (2026, 37), (8101,), (2026,))

In [28]:
#GRID SEARCH AND CROSS VALIDATION
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import svm

In [50]:
#SVM 
#DEFINE PARAMETERS
# Parameters={'Kernel':['linear', 'rbf'], 'C':[1,10,20] }
SVC=svm.SVC
# GridSearch=GridSearchCV(SVC, Parameters, scoring='accuracy', cv=3)
param_grid = { 'C':[0.1,1,100],'kernel':['rbf','poly','sigmoid','linear'],
              'degree':[2,3,4,5],'gamma': [1, 0.1, 0.01, 0.001]}
GridSearch = GridSearchCV(SVC(),param_grid, cv=3, verbose=2)

In [51]:
GridSearch.fit(XTrain, YTrain)

Fitting 3 folds for each of 192 candidates, totalling 576 fits
[CV] END ...............C=0.1, degree=2, gamma=1, kernel=rbf; total time=   4.4s
[CV] END ...............C=0.1, degree=2, gamma=1, kernel=rbf; total time=   4.5s
[CV] END ...............C=0.1, degree=2, gamma=1, kernel=rbf; total time=   4.4s
[CV] END ..............C=0.1, degree=2, gamma=1, kernel=poly; total time=   0.8s
[CV] END ..............C=0.1, degree=2, gamma=1, kernel=poly; total time=   0.8s
[CV] END ..............C=0.1, degree=2, gamma=1, kernel=poly; total time=   0.9s
[CV] END ...........C=0.1, degree=2, gamma=1, kernel=sigmoid; total time=   1.3s
[CV] END ...........C=0.1, degree=2, gamma=1, kernel=sigmoid; total time=   1.2s
[CV] END ...........C=0.1, degree=2, gamma=1, kernel=sigmoid; total time=   1.0s
[CV] END ............C=0.1, degree=2, gamma=1, kernel=linear; total time=   0.5s
[CV] END ............C=0.1, degree=2, gamma=1, kernel=linear; total time=   0.4s
[CV] END ............C=0.1, degree=2, gamma=1,

[CV] END ..............C=0.1, degree=4, gamma=1, kernel=poly; total time=   0.6s
[CV] END ...........C=0.1, degree=4, gamma=1, kernel=sigmoid; total time=   1.5s
[CV] END ...........C=0.1, degree=4, gamma=1, kernel=sigmoid; total time=   1.4s
[CV] END ...........C=0.1, degree=4, gamma=1, kernel=sigmoid; total time=   1.0s
[CV] END ............C=0.1, degree=4, gamma=1, kernel=linear; total time=   0.4s
[CV] END ............C=0.1, degree=4, gamma=1, kernel=linear; total time=   0.4s
[CV] END ............C=0.1, degree=4, gamma=1, kernel=linear; total time=   0.4s
[CV] END .............C=0.1, degree=4, gamma=0.1, kernel=rbf; total time=   1.4s
[CV] END .............C=0.1, degree=4, gamma=0.1, kernel=rbf; total time=   1.6s
[CV] END .............C=0.1, degree=4, gamma=0.1, kernel=rbf; total time=   1.5s
[CV] END ............C=0.1, degree=4, gamma=0.1, kernel=poly; total time=   0.6s
[CV] END ............C=0.1, degree=4, gamma=0.1, kernel=poly; total time=   0.5s
[CV] END ............C=0.1, 

[CV] END ..............C=1, degree=2, gamma=1, kernel=linear; total time=   0.6s
[CV] END ...............C=1, degree=2, gamma=0.1, kernel=rbf; total time=   1.2s
[CV] END ...............C=1, degree=2, gamma=0.1, kernel=rbf; total time=   1.2s
[CV] END ...............C=1, degree=2, gamma=0.1, kernel=rbf; total time=   1.1s
[CV] END ..............C=1, degree=2, gamma=0.1, kernel=poly; total time=   0.4s
[CV] END ..............C=1, degree=2, gamma=0.1, kernel=poly; total time=   0.4s
[CV] END ..............C=1, degree=2, gamma=0.1, kernel=poly; total time=   0.4s
[CV] END ...........C=1, degree=2, gamma=0.1, kernel=sigmoid; total time=   0.4s
[CV] END ...........C=1, degree=2, gamma=0.1, kernel=sigmoid; total time=   0.4s
[CV] END ...........C=1, degree=2, gamma=0.1, kernel=sigmoid; total time=   0.4s
[CV] END ............C=1, degree=2, gamma=0.1, kernel=linear; total time=   0.6s
[CV] END ............C=1, degree=2, gamma=0.1, kernel=linear; total time=   0.6s
[CV] END ............C=1, de

[CV] END ..............C=1, degree=4, gamma=0.1, kernel=poly; total time=   0.5s
[CV] END ...........C=1, degree=4, gamma=0.1, kernel=sigmoid; total time=   0.4s
[CV] END ...........C=1, degree=4, gamma=0.1, kernel=sigmoid; total time=   0.4s
[CV] END ...........C=1, degree=4, gamma=0.1, kernel=sigmoid; total time=   0.4s
[CV] END ............C=1, degree=4, gamma=0.1, kernel=linear; total time=   0.6s
[CV] END ............C=1, degree=4, gamma=0.1, kernel=linear; total time=   0.5s
[CV] END ............C=1, degree=4, gamma=0.1, kernel=linear; total time=   0.5s
[CV] END ..............C=1, degree=4, gamma=0.01, kernel=rbf; total time=   1.1s
[CV] END ..............C=1, degree=4, gamma=0.01, kernel=rbf; total time=   0.9s
[CV] END ..............C=1, degree=4, gamma=0.01, kernel=rbf; total time=   1.0s
[CV] END .............C=1, degree=4, gamma=0.01, kernel=poly; total time=   0.5s
[CV] END .............C=1, degree=4, gamma=0.01, kernel=poly; total time=   0.5s
[CV] END .............C=1, d

[CV] END ..........C=100, degree=2, gamma=0.1, kernel=linear; total time=  14.3s
[CV] END ............C=100, degree=2, gamma=0.01, kernel=rbf; total time=   1.0s
[CV] END ............C=100, degree=2, gamma=0.01, kernel=rbf; total time=   0.9s
[CV] END ............C=100, degree=2, gamma=0.01, kernel=rbf; total time=   0.9s
[CV] END ...........C=100, degree=2, gamma=0.01, kernel=poly; total time=   0.4s
[CV] END ...........C=100, degree=2, gamma=0.01, kernel=poly; total time=   0.3s
[CV] END ...........C=100, degree=2, gamma=0.01, kernel=poly; total time=   0.3s
[CV] END ........C=100, degree=2, gamma=0.01, kernel=sigmoid; total time=   0.3s
[CV] END ........C=100, degree=2, gamma=0.01, kernel=sigmoid; total time=   0.3s
[CV] END ........C=100, degree=2, gamma=0.01, kernel=sigmoid; total time=   0.3s
[CV] END .........C=100, degree=2, gamma=0.01, kernel=linear; total time=  13.3s
[CV] END .........C=100, degree=2, gamma=0.01, kernel=linear; total time=  14.5s
[CV] END .........C=100, deg

[CV] END ...........C=100, degree=4, gamma=0.01, kernel=poly; total time=   0.5s
[CV] END ........C=100, degree=4, gamma=0.01, kernel=sigmoid; total time=   0.3s
[CV] END ........C=100, degree=4, gamma=0.01, kernel=sigmoid; total time=   0.3s
[CV] END ........C=100, degree=4, gamma=0.01, kernel=sigmoid; total time=   0.3s
[CV] END .........C=100, degree=4, gamma=0.01, kernel=linear; total time=  13.3s
[CV] END .........C=100, degree=4, gamma=0.01, kernel=linear; total time=  14.6s
[CV] END .........C=100, degree=4, gamma=0.01, kernel=linear; total time=  14.3s
[CV] END ...........C=100, degree=4, gamma=0.001, kernel=rbf; total time=   0.9s
[CV] END ...........C=100, degree=4, gamma=0.001, kernel=rbf; total time=   0.9s
[CV] END ...........C=100, degree=4, gamma=0.001, kernel=rbf; total time=   0.9s
[CV] END ..........C=100, degree=4, gamma=0.001, kernel=poly; total time=   0.4s
[CV] END ..........C=100, degree=4, gamma=0.001, kernel=poly; total time=   0.4s
[CV] END ..........C=100, de

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 1, 100], 'degree': [2, 3, 4, 5],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             verbose=2)

In [53]:
GridSearch.best_params_
#{'C': 100, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}

{'C': 100, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}

In [54]:
YPred=GridSearch.predict(XTest)

In [36]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score

In [55]:
confusion_matrix(YPred, YTest)

array([[1651,   67],
       [  49,  259]])

In [56]:
accuracy_score(YPred, YTest), precision_score(YPred, YTest), recall_score(YPred, YTest)

(0.9427443237907206, 0.7944785276073619, 0.8409090909090909)

In [68]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [57]:
#Grid Search Parameters for RF

# Number of trees in random forest
n_estimators=[100, 400, 1000]

# Maximum number of levels in tree
max_depth=[10, 30, 100]

# Number of features to consider at every split
max_features=['auto', 'sqrt']

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]


# BEST ONE SO FAR
# {'n_estimators': 400,
#  'min_samples_split': 5,
#  'min_samples_leaf': 1,
#  'max_features': 'auto',
#  'max_depth': 80,
#  'bootstrap': False}



In [58]:
RFParameterGrid={'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [63]:
RFClf=RandomForestClassifier(random_state=61)

In [69]:
RFGridSearch=RandomizedSearchCV(RFClf, RFParameterGrid, verbose=2, n_iter = 100)

In [70]:
RFGridSearch.fit(XTrain, YTrain)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   3.5s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   3.4s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   3.4s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   3.4s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   3.4s
[CV] END bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1000; total time=   6.7s
[CV] END bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimator

[CV] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1000; total time=  12.3s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=400; total time=   3.8s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=400; total time=   3.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=400; total time=   3.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=400; total time=   3.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=400; total time=   3.0s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   1.2s
[CV] END bootstrap=False, max_depth=80, max

[CV] END bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.8s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.0s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.9s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.9s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.9s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.0s
[CV] END bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1000; total time=   7.8s
[CV] END bootstrap=True, max_depth=100,

[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   4.4s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   2.1s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   2.1s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   2.0s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   2.0s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   2.2s
[CV] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   1.1s
[CV] END bootstrap=False, ma

[CV] END bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.7s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=400; total time=   3.9s
[CV] END bootstrap=False, max_depth=100,

[CV] END bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=400; total time=   2.6s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   2.4s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   2.3s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   2.4s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   2.3s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   2.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   2.3s
[CV] END bootstrap=True, max_depth=10, max_fea

[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   6.5s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   5.9s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   5.8s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   5.9s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   5.8s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   5.9s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   1.3s
[CV] END bootstrap=True, max_depth=30, m

[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   3.7s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.9s
[CV] END bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END bootstrap=True, max_depth=80, ma

[CV] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.9s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=400; total time=   3.7s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=400; total time=   3.6s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=400; total time=   3.7s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=400; total time=   3.6s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=400; total time=   3.7s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   3.2s
[CV] END bootstrap=False, max_depth=10,

RandomizedSearchCV(estimator=RandomForestClassifier(random_state=61),
                   n_iter=100,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 30, 80, 100],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 400, 1000]},
                   verbose=2)

In [71]:
RFGridSearch.best_params_

{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 80,
 'bootstrap': False}

In [72]:
RFYPredict=RFGridSearch.predict(XTest)

In [73]:
confusion_matrix(RFYPredict, YTest)

array([[1676,   47],
       [  24,  279]])

In [74]:
accuracy_score(RFYPredict, YTest), precision_score(RFYPredict, YTest), recall_score(RFYPredict, YTest)

(0.9649555774925962, 0.8558282208588958, 0.9207920792079208)

In [ ]:
#LEARN FROM THIS
#https://www.kaggle.com/vivek468/will-the-customer-churn/data
#https://www.kaggle.com/andreshg/automatic-eda-libraries-comparisson#3.-%F0%9F%93%8A-Pandas-Profiling-%F0%9F%93%9A